In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from scipy.signal import find_peaks
#from sklearn.neighbors import KernelDensity

In [2]:
gyro = pd.read_csv("data/android.sensor.gyroscope.csv")
accel = pd.read_csv("data/android.sensor.accelerometer.csv")
beats = [float(line.rstrip()) for line in open("data/beats.txt")]

In [3]:
accel_time = accel.iloc[:,0] / 1000000000
gyro_time = gyro.iloc[:,0] / 1000000000
offset_seconds = 11.541678

In [29]:
accel_peaks_x_samples , temp = find_peaks(accel.iloc[:,1],prominence=(25,None))
accel_peaks_x_prominence = temp["prominences"]
accel_peaks_x = accel_time[accel_peaks_x_samples]

accel_valleys_x_samples , temp = find_peaks(-1*accel.iloc[:,1],prominence=(25,None))
accel_valleys_x_prominence = temp["prominences"]
accel_valleys_x = accel_time[accel_valleys_x_samples]

accel_peaks_y_samples , temp = find_peaks(accel.iloc[:,2],prominence=(25,None))
accel_peaks_y_prominence = temp["prominences"]
accel_peaks_y = accel_time[accel_peaks_y_samples]

accel_valleys_y_samples , temp = find_peaks(-1*accel.iloc[:,2],prominence=(25,None))
accel_valleys_y_prominence = temp["prominences"]
accel_valleys_y = accel_time[accel_valleys_y_samples]

accel_peaks_z_samples , temp = find_peaks(accel.iloc[:,3],prominence=(25,None))
accel_peaks_z_prominence = temp["prominences"]
accel_peaks_z = accel_time[accel_peaks_z_samples]

accel_valleys_z_samples , temp = find_peaks(-1*accel.iloc[:,3],prominence=(25,None))
accel_valleys_z_prominence = temp["prominences"]
accel_valleys_z = accel_time[accel_valleys_z_samples]

gyro_peaks_x_samples , temp = find_peaks(gyro.iloc[:,1],prominence=(None,None))
gyro_peaks_x_prominence = temp["prominences"]
gyro_peaks_x = gyro_time[gyro_peaks_x_samples]

gyro_valleys_x_samples , temp = find_peaks(-1*gyro.iloc[:,1],prominence=(None,None))
gyro_valleys_x_prominence = temp["prominences"]
gyro_valleys_x = gyro_time[gyro_valleys_x_samples]

gyro_peaks_y_samples , temp = find_peaks(gyro.iloc[:,2],prominence=(None,None))
gyro_peaks_y_prominence = temp["prominences"]
gyro_peaks_y = gyro_time[gyro_peaks_y_samples]

gyro_valleys_y_samples , temp = find_peaks(-1*gyro.iloc[:,2],prominence=(None,None))
gyro_valleys_y_prominence = temp["prominences"]
gyro_valleys_y = gyro_time[gyro_valleys_y_samples]

gyro_peaks_z_samples , temp = find_peaks(gyro.iloc[:,3],prominence=(None,None))
gyro_peaks_z_prominence = temp["prominences"]
gyro_peaks_z = gyro_time[gyro_peaks_z_samples]

gyro_valleys_z_samples , temp = find_peaks(-1*gyro.iloc[:,3],prominence=(None,None))
gyro_valleys_z_prominence = temp["prominences"]
gyro_valleys_z = gyro_time[gyro_valleys_z_samples]

Shows the acceleration over time split by axis and with peaks with a prominence over 25

In [38]:
plt.close()
fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

fig.suptitle('acceleration with peaks with prominence > 25', fontsize=16)

fig.supxlabel("time in seconds")
fig.supylabel("acceleration in m/s^2")

for x in beats:
    seconds = x+ offset_seconds

    axs[0].axvline(seconds, color = 'g')
    axs[1].axvline(seconds, color = 'g')
    axs[2].axvline(seconds, color = 'g')


axs[0].plot(accel_peaks_x, np.array(accel.iloc[accel_peaks_x_samples,1]), "x")
axs[0].plot(accel_valleys_x, np.array(accel.iloc[accel_valleys_x_samples,1]), "o")
axs[0].plot(accel_time,accel.iloc[:,1], color = 'r')
axs[0].grid(True)
axs[0].set_title("x")


axs[1].plot(accel_peaks_y, np.array(accel.iloc[accel_peaks_y_samples,2]), "x")
axs[1].plot(accel_valleys_y, np.array(accel.iloc[accel_valleys_y_samples,2]), "o")
axs[1].plot(accel_time,accel.iloc[:,2], color = 'r')
axs[1].grid(True)
axs[1].set_title("y")


axs[2].plot(accel_peaks_z, np.array(accel.iloc[accel_peaks_z_samples,3]), "x")
axs[2].plot(accel_valleys_z, np.array(accel.iloc[accel_valleys_z_samples,3]), "o")
axs[2].plot(accel_time,accel.iloc[:,3], color = 'r')
axs[2].grid(True)
axs[2].set_title("z")


plt.grid(True)
plt.show()

In [5]:
def find_most_prominent_peak(peaks,peaks_prominences,valleys,valleys_prominences,min_threshold = 0,max_treshold = inf):
    max_prominence = 0
    max_peak = 0
    offset = (min_threshold + max_treshold) / 2
    for peak, prominence in zip(peaks, peaks_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    for peak, prominence in zip(valleys, valleys_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    return (max_peak, max_prominence)
    
def calc_prominence_factor(x,threshold = 0.2,offset=0):
    return exp(0-100*(x-offset)**2)

In [52]:
min_threshold = 0
max_threshold = inf

beat_diff = (beats[0] + beats[-1])/ (len(beats)-1)
threshold = beat_diff / 2

accel_peaks = []
accel_peaks_axis = []
gyro_peaks = []

for beat, i in zip(beats,range(len(beats))):

    #if i < (len(beats)-1):
    #    max_threshold = (beats[i+1] + beat) / 2
    #else: 
    #    max_threshold = inf

    min_threshold = beat - threshold
    max_threshold = beat + threshold

    accel_peaks_with_prominence = []
    gyro_peaks_with_prominence = []


    
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_x,accel_peaks_x_prominence,accel_valleys_x,accel_valleys_x_prominence,min_threshold, max_threshold))
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_y,accel_peaks_y_prominence,accel_valleys_y,accel_valleys_y_prominence,min_threshold, max_threshold))
    accel_peaks_with_prominence.append(find_most_prominent_peak(accel_peaks_z,accel_peaks_z_prominence,accel_valleys_z,accel_valleys_z_prominence,min_threshold, max_threshold))

    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_x,gyro_peaks_x_prominence,gyro_valleys_x,gyro_valleys_x_prominence,min_threshold, max_threshold))
    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_y,gyro_peaks_y_prominence,gyro_valleys_y,gyro_valleys_y_prominence,min_threshold, max_threshold))
    gyro_peaks_with_prominence.append(find_most_prominent_peak(gyro_peaks_z,gyro_peaks_z_prominence,gyro_valleys_z,gyro_valleys_z_prominence,min_threshold, max_threshold))

    max_accel_prominence = 0
    max_peak_position = 0
    for peak, prominence in accel_peaks_with_prominence:
        if prominence > max_accel_prominence:
            max_accel_prominence = prominence
            max_peak_position = peak

    accel_peak_sum = 0
    accel_peak_count = 0
    axis = []


    
    for (peak, prominence), index in zip(accel_peaks_with_prominence,range(3)):
        if prominence  >= (max_accel_prominence * 0.75) and abs(peak-max_peak_position) < threshold / 4:
            accel_peak_sum += peak
            accel_peak_count += 1
            axis.append(index)
    if accel_peak_sum > 0:
        accel_peak = accel_peak_sum / accel_peak_count
        accel_peaks.append((accel_peak,max_accel_prominence,accel_peak_count,beat))
        accel_peaks_axis.append(axis)

    min_threshold = max_threshold


Shows the time diffrent each motionbeat has to its next musicbeat

In [70]:
plt.close()
plt.axhline(beat_diff)
#plt.axhline(threshold)
plt.xlabel("time in seconds")
plt.ylabel("distance to next beat in seconds")
plt.title("time delta for each beat")
#plt.axhline(-1*beat_diff)
#plt.axhline(-1*threshold)
for peak, prominence , _ ,beat in accel_peaks:
    diff = abs(peak - beat)
    plt.plot(beat, diff,"g.", label="beat")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.close()
fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

fig.suptitle('acceleration with motion- and music beats', fontsize=16)

fig.supxlabel("time in seconds")
fig.supylabel("acceleration in m/s^2")

for i in range(3):
    axs[i].axvline(beats[0] + offset_seconds, color = 'g',label = "music beat")
    axs[i].legend()

    for peak, axis in zip(accel_peaks, accel_peaks_axis):
        if i in axis:
            axs[i].axvline(peak[0],color='b',label = "motion beat")
            break
#axs[accel_peaks_axis[0][0]].axvline(accel_peaks[0][0],color='b',label = "b")


for x in beats:
    seconds = x+ offset_seconds

    axs[0].axvline(seconds, color = 'g')
    axs[1].axvline(seconds, color = 'g')
    axs[2].axvline(seconds, color = 'g')

for peak, axis in zip(accel_peaks, accel_peaks_axis):
    for i in axis:
        axs[i].axvline(peak[0], color = 'b')


axs[0].plot(accel_peaks_x, np.array(accel.iloc[accel_peaks_x_samples,1]), "x")
axs[0].plot(accel_valleys_x, np.array(accel.iloc[accel_valleys_x_samples,1]), "o")
axs[0].plot(accel_time,accel.iloc[:,1], color = 'r')
axs[0].grid(True)
axs[0].set_title("x")


axs[1].plot(accel_peaks_y, np.array(accel.iloc[accel_peaks_y_samples,2]), "x")
axs[1].plot(accel_valleys_y, np.array(accel.iloc[accel_valleys_y_samples,2]), "o")
axs[1].plot(accel_time,accel.iloc[:,2], color = 'r')
axs[1].grid(True)
axs[1].set_title("y")


axs[2].plot(accel_peaks_z, np.array(accel.iloc[accel_peaks_z_samples,3]), "x")
axs[2].plot(accel_valleys_z, np.array(accel.iloc[accel_valleys_z_samples,3]), "o")
axs[2].plot(accel_time,accel.iloc[:,3], color = 'r')
axs[2].grid(True)
axs[2].set_title("z")


plt.grid(True)
axs[0].legend()
plt.show()